In [1]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, concatenate, GlobalAveragePooling2D, Dense, MaxPooling2D, Dropout, Flatten, ReLU, Add, BatchNormalization, AveragePooling2D, Multiply, GlobalAveragePooling2D, Reshape,DepthwiseConv2D
from keras.optimizers import RMSprop
import keras.backend as K
import cv2
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from PIL import Image
from tensorflow.keras.metrics import Metric,Precision, Recall
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, MobileNet
import pandas as pd

In [2]:
train_dir = r'E:\MV\coursework\fruits-360\Training'
test_dir = r'E:\MV\coursework\fruits-360\Test'
#The total number of images: 90483.
#Training set size: 67692 images (one fruit or vegetable per image).
#Test set size: 22688 images (one fruit or vegetable per image).
#The number of classes: 131 (fruits and vegetables).
#Image size: 100x100 pixels.

In [3]:
#find out the total calsses of dataset.
className = glob.glob(train_dir + "/*")
numberOfClass = len(className)
print("NumberOfClass: ",numberOfClass)

NumberOfClass:  131


In [4]:
def denoise_background(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    thresh_rgb = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    np.copyto(image, thresh_rgb, where=thresh_rgb == 255)
    return image

In [5]:
# Load and preprocess the training data
train_datagen = ImageDataGenerator(
    preprocessing_function=denoise_background,
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    validation_split=0.068,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb', 
    subset='training' 
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb', 
    subset='validation'
)

# Load and preprocess the test data
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

Found 63145 images belonging to 131 classes.
Found 4547 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [6]:
input_shape = Input(shape=(100,100,3))

In [7]:
class CustomAccuracy(Metric):
    def __init__(self, name='accuracy', **kwargs):
        super(CustomAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight(name='correct', initializer='zeros')
        self.total = self.add_weight(name='total', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_classes = tf.argmax(y_pred, axis=1)
        y_true_classes = tf.argmax(y_true, axis=1)
        correct_predictions = tf.equal(y_pred_classes, y_true_classes)
        correct_predictions = tf.cast(correct_predictions, 'float32')
        
        self.correct.assign_add(tf.reduce_sum(correct_predictions))
        self.total.assign_add(tf.cast(tf.size(y_true_classes), 'float32'))

    def result(self):
        return tf.divide(self.correct, self.total)

    def reset_state(self):
        self.correct.assign(0)
        self.total.assign(0)


In [8]:
class CustomCrossEntropyLoss(tf.keras.losses.Loss):
    def __init__(self, name='loss', **kwargs):
        super(CustomCrossEntropyLoss, self).__init__(name=name, **kwargs)

    def call(self, y_true, y_pred):
        epsilon = 1e-10
        y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)

        loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

        return loss

Add the output layer for the other models

In [11]:
def add_new_last_layer(base_model):
    x = Dense(128, activation='relu')(base_model.output) 
    x = BatchNormalization()(x) 
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x) 
    x = Dropout(0.3)(x)
    last_output= Dense(numberOfClass, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=last_output)
    return model 

def prepare_model(model):
    model = add_new_last_layer(model)
    model.compile(optimizer=RMSprop(learning_rate=0.001), loss=CustomCrossEntropyLoss(),metrics=[CustomAccuracy(), Precision(name='precision'),Recall(name='recall')])
    return model

In [12]:
resnet50_model = prepare_model(ResNet50(weights=None, include_top=False, input_shape=(100,100,3)))

Train the combined model

In [14]:
history_resnet50=resnet50_model.fit(train_generator, epochs=40, validation_data=validation_generator)

Epoch 1/40
1974/1974 [==============================] - 519s 256ms/step - loss: 1.5713 - accuracy: 0.6242 - val_loss: 0.6700 - val_accuracy: 0.8276
Epoch 2/40
1974/1974 [==============================] - 337s 171ms/step - loss: 0.2249 - accuracy: 0.9405 - val_loss: 0.3250 - val_accuracy: 0.9197
Epoch 3/40
1974/1974 [==============================] - 336s 170ms/step - loss: 0.1369 - accuracy: 0.9658 - val_loss: 0.2364 - val_accuracy: 0.9384
Epoch 4/40
1974/1974 [==============================] - 337s 171ms/step - loss: 0.1034 - accuracy: 0.9756 - val_loss: 4.3163 - val_accuracy: 0.4121
Epoch 5/40
1974/1974 [==============================] - 336s 170ms/step - loss: 0.0836 - accuracy: 0.9812 - val_loss: 0.4808 - val_accuracy: 0.9070
Epoch 6/40
1974/1974 [==============================] - 336s 170ms/step - loss: 0.0723 - accuracy: 0.9839 - val_loss: 0.5229 - val_accuracy: 0.9074
Epoch 7/40
1974/1974 [==============================] - 336s 170ms/step - loss: 0.0591 - accuracy: 0.9870 - val_

In [30]:
#save the history of mobilenet    
save_history_resnet50 = pd.DataFrame(history_resnet50.history)

save_history_resnet50.to_csv("resnet50_model_training_history_40epoch_1211.csv", index=False)

save_path=r'resnet50_model1213_40epochs.h5'
resnet50_model.save(save_path)